## OpenAI api

In [1]:
# text cleaning function (with lemitization and tokenization)
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer
import re
def preprocess_text( text):
       text = text.lower()                             # all lower case
       text = re.sub(r'\[.*?\]', ' ', text)            # remove text within [ ] (' ' instead of '')
       text = re.sub(r'\<.*?\>', ' ', text)            # remove text within < > (' ' instead of '')
       text = re.sub(r'http\S+', ' ', text)            # remove website ref http
       text = re.sub(r'www\S+', ' ', text)             # remove website ref www

       text = text.replace('€', 'euros')               # replace special character with words
       text = text.replace('£', 'gbp')                 # replace special character with words
       text = text.replace('$', 'dollar')              # replace special character with words
       text = text.replace('%', 'percent')             # replace special character with words
       text = text.replace('\n', ' ')                  # remove \n in text that has it

       text = text.replace('\'', '’')                  # standardise apostrophe
       text = text.replace('&#39;', '’')               # standardise apostrophe

       text = text.replace('’d', ' would')             # remove ’ (for would, should? could? had + PP?)
       text = text.replace('’s', ' is')                # remove ’ (for is, John's + N?)
       text = text.replace('’re', ' are')              # remove ’ (for are)
       text = text.replace('’ll', ' will')             # remove ’ (for will)
       text = text.replace('’ve', ' have')             # remove ’ (for have)
       text = text.replace('’m', ' am')                # remove ’ (for am)
       text = text.replace('can’t', 'can not')         # remove ’ (for can't)
       text = text.replace('won’t', 'will not')        # remove ’ (for won't)
       text = text.replace('n’t', ' not')              # remove ’ (for don't, doesn't)

       text = text.replace('’', ' ')                   # remove apostrophe (in general)
       text = text.replace('&quot;', ' ')              # remove quotation sign (in general)

       text = text.replace('cant', 'can not')          # typo 'can't' (note that cant is a proper word)
       text = text.replace('dont', 'do not')           # typo 'don't'

       text = re.sub(r'[^a-zA-Z0-9]', r' ', text)      # only alphanumeric left
       text = text.replace("   ", ' ')                 # remove triple empty space
       text = text.replace("  ", ' ')                  # remove double empty space

       tokens = TreebankWordTokenizer().tokenize(text)
       tokens = list(map(WordNetLemmatizer().lemmatize, tokens))
       preprocessed_text = " ".join(tokens)

       return preprocessed_text

In [3]:
# import libraries
import os
import openai
import time

openai.api_key = os.getenv("OPENAI_API_KEY")

In [6]:
# qns to ask (metrics and sustainability)
qns = ["what is the metrics for designing sustainable heater?","how to design a sustainable heater?","what makes an enery efficient heater?"]

In [53]:
# OpenAI 'chat' model
ans = []

for q in qns:  
  response = openai.Completion.create(
    engine="text-davinci-002",
    prompt=q,
    temperature=0,
    max_tokens=150,
    top_p=0,
    frequency_penalty=2,
    presence_penalty=2,
    stop=[" Human:", " AI:"]
  )
  response_text = response.choices[0]['text']
  response_text = response_text.replace("\n", " ")
  time.sleep(0.1)
  try:
    response_text = response_text.split(':', 1)[1] 
  except:
    response_text = response_text.split('.', 1)[1] 
  finally:
    response_text = preprocess_text(response_text)
    print('questions: ', q)
    print('answer: ',response_text)
    ans.append(response_text)
  

questions:  what is the metrics for designing sustainable heater?
answer:  however some possible metric that could be used to design a sustainable heater include energy efficiency measured in term of british thermal unit or btu emission output measured in term of carbon dioxide equivalent or co2e water conservation measured in gallon per minute or gpm and material reuse recycling potential
questions:  how to design a sustainable heater?
answer:  however some tip for designing a sustainable heater include using renewable energy source such a solar or wind power incorporating thermal mass into the design to help store heat and using insulation to minimize heat loss
questions:  what makes an enery efficient heater?
answer:  the type of fuel being used e g natural gas electricity propane the efficiency rating of the appliance and how well insulated the space is that need to be heated


In [55]:
# extract the key terms from the answer related to designing sustainable heater and its metrics
ans_key = []
for a in ans:
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt="Extract keywords from this text:\n\n{}".format(a),
        temperature=0.8, # for more desire results ( 0 is exact words, 1 is more general)
        max_tokens=40,
        top_p=1,
        frequency_penalty=1,
        presence_penalty=1
    )
    response_text = response.choices[0]['text']
    response_text = response_text.replace("\n", " ").replace("  ", "")
    ans_key.append(response_text)

ans_key
    

['energy efficiency, btu, emission output, carbon dioxide equivalent, water conservation, gallon per minute, material reuse',
 'sustainable, heater, design, renewable, energy, source, solar, wind, power, thermal mass',
 'fuel, efficiency, appliance, space, heated']

In [56]:
# filter key terms that not repeated in questions
qns_key = []
for q in qns:
    q = q.replace("heater?", "heater")
    qns_key.extend(q.split(' '))

ans_key_clean = []
for a_string in ans_key:
    a_list = a_string.split(', ')
   
    for a in a_list:
        if a not in qns_key and 'efficien' not in a and 'sustainable' not in a and 'energy' not in a and 'heater' not in a:
            ans_key_clean.append(a)
print('key terms: ', ans_key_clean)


key terms:  ['btu', 'emission output', 'carbon dioxide equivalent', 'water conservation', 'gallon per minute', 'material reuse', 'renewable', 'source', 'solar', 'wind', 'power', 'thermal mass', 'fuel', 'appliance', 'space', 'heated']


In [57]:
# qns to ask (functional, behaviour, structure)
qns2 = ["functional design of heater?",'behavioral design for heater?', 'structural design of heater?']

In [58]:
# OpenAI 'chat' model
ans2 = []
for q in qns2:  
  response = openai.Completion.create(
    engine="text-davinci-002",
    prompt=q,
    temperature=0,
    max_tokens=150,
    top_p=1,
    frequency_penalty=1,
    presence_penalty=1
  )
  time.sleep(2)
  response_text = response.choices[0]['text']
  response_text = response_text.replace("\n", " ")
  time.sleep(0.1)
  try:
    response_text = response_text.split(':', 1)[1] 
  except:
    response_text = response_text.split('.', 1)[1] 
  finally:
    response_text = preprocess_text(response_text)
    print('question: ', q)
    print('answer: ' ,response_text)
    ans2.append(response_text)
    time.sleep(2)
  

question:  functional design of heater?
answer:  the user should also be able to easily set the timer for when the heater will turn on and off
question:  behavioral design for heater?
answer:  however some tip for designing a heater with good behavioral property include making sure that the heater is easy to use and control providing clear feedback to the user about the current temperature and setting and incorporating feature that encourage energy efficiency
question:  structural design of heater?
answer:  the most common type of heater is the furnace which us hot air to warm the interior of a home


In [59]:
# OpenAI 'TL;DR' model
response = openai.Completion.create(
  engine="text-davinci-002",
  prompt= ''.join(ans2) + "\n\nTl;dr:\n\n",
  temperature=0.5, # for more desire results ( 0 is exact words, 1 is more general)
  max_tokens=500,
  top_p=1,
  frequency_penalty=2,
  presence_penalty=2
)
response_text_summary2 = response.choices[0]['text'].replace("\n", " ")
print('Short summary of what is important for designing heater: ')
print(response_text_summary2)

Short summary of what is important for designing heater: 
  The most important thing to consider when designing a heater is how it will be used and what the user's needs are. Some tips for designing a good heater include making sure that it is easy to use and control, providing clear feedback about the current temperature and setting, and incorporating features that encourage energy efficiency. The most common type of heater is the furnace, which uses hot air to warm the interior of a home.


In [60]:
#extract important keywords (Functional, Behaviour and Structural Design)
ans_key2 = []
for a in ans2:
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt="Extract keywords from this text:\n\n{}".format(a),
        temperature=0,
        max_tokens=60,
        top_p=1,
        frequency_penalty=1.1,
        presence_penalty=1.2
    )
    response_text = response.choices[0]['text']
    response_text = response_text.replace("\n", " ").replace("  ", "")
    ans_key2.append(response_text)
    time.sleep(2)

ans_key2
ans_key2_clean = []
for a_string in ans_key2:
    a_list = a_string.split(', ')
   
    for a in a_list:
        if a not in qns_key and 'efficien' not in a and 'sustainable' not in a and 'energy' not in a and 'eater' not in a:
            ans_key2_clean.append(a)
print('from: ',ans2)
print('to: ',ans_key2_clean)
    

from:  ['the user should also be able to easily set the timer for when the heater will turn on and off', 'however some tip for designing a heater with good behavioral property include making sure that the heater is easy to use and control providing clear feedback to the user about the current temperature and setting and incorporating feature that encourage energy efficiency', 'the most common type of heater is the furnace which us hot air to warm the interior of a home']
to:  ['timer', 'on', 'off.', 'behavioral property', 'easy to use', 'control', 'feedback', 'temperature', 'setting', 'furnace', 'hot air', 'warm', 'interior', 'home.']


In [62]:
# qns to ask (sustainability)
qns3 = ['designing for sustainable heater?','designing for efficient heater?','designing for efficient and sustainable heater?']

In [65]:
# get AI to answer how to design questions
ans3 = []

for q in qns3:  
    response = openai.Completion.create(
    engine="text-davinci-002",
    prompt="What are 10 key points I should know when "+q,
    temperature=0,
    max_tokens=150,
    top_p=1,
    frequency_penalty=1,
    presence_penalty=1
    )
    response_text = response.choices[0]['text']
    response_text = response_text.replace("\n\n", ",")
    time.sleep(0.1)
  
    response_text = preprocess_text(response_text)
    s = re.sub("^\d+\s|\s\d+\s|\s\d+$", ",", response_text)
    
    ans3.append(s[1:])

print('questions on designning a sustainable heater: ')
print(ans3)

questions on designning a sustainable heater: 
['sustainability is important when designing a heater,the most efficient heater are those that use the least amount of energy to produce heat,solar powered heater are the most sustainable option a they rely on renewable energy from the sun,electric resistance heater are also quite sustainable a they do not produce any emission,heat pump are another efficient and sustainable option for heating your home or office space,geothermal system are yet another type of sustainable heater a they use the earth is natural heat to warm your space,when choosing a sustainable heater it is important to consider the upfront cost operating cost and maintenance cost over the', 'make sure the heater is properly sized for the space it will be used in,place the heater in a central location so that it can evenly heat the space,insulate the wall and ceiling around the heater to help keep heat in,use drapery or blind to cover window and help keep heat in,keep door 

In [80]:
# attemp1 to extract key terms
ans_key3 = []
for a in ans3:
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt="Extract keywords from this text:\n\n{}".format(a),
        temperature=0,
        max_tokens=100,
        top_p=1,
        frequency_penalty=1,
        presence_penalty=1
    )
    response_text = response.choices[0]['text']
    response_text = response_text.replace("\n", " ")
    ans_key3.append(response_text)

qns_key3 = []
for q in qns3:
    q = q.replace("heater?", "heater")
    qns_key3.extend(q.split(' '))

ans_key3_clean = []
for a_string in ans_key3:
    a_list = a_string.split(', ')
   
    for a in a_list:
        if a not in qns_key3:
            if 'efficien' not in a and 'sustain' not in a and 'eate' not in a and 'renew' not in a and 'energy' not in a:
                ans_key3_clean.append(a)
print('from: ',ans3)
print('to:' ,ans_key3_clean)

from:  ['sustainability is important when designing a heater,the most efficient heater are those that use the least amount of energy to produce heat,solar powered heater are the most sustainable option a they rely on renewable energy from the sun,electric resistance heater are also quite sustainable a they do not produce any emission,heat pump are another efficient and sustainable option for heating your home or office space,geothermal system are yet another type of sustainable heater a they use the earth is natural heat to warm your space,when choosing a sustainable heater it is important to consider the upfront cost operating cost and maintenance cost over the', 'make sure the heater is properly sized for the space it will be used in,place the heater in a central location so that it can evenly heat the space,insulate the wall and ceiling around the heater to help keep heat in,use drapery or blind to cover window and help keep heat in,keep door closed to room that are not being heated

In [82]:
# let's turn it to Summary first before extracting key terms
response = openai.Completion.create(
  engine="text-davinci-002",
  prompt= "when designing a heater" + ''.join(ans3) + "\n\nTl;dr:\n\n",
  temperature=0,
  max_tokens=250,
  top_p=1,
  frequency_penalty=1,
  presence_penalty=1
)
response_text_summary = response.choices[0]['text'].replace("\n", "")
response_text_summary

"When designing a heater, sustainability is important. The most efficient heaters are those that use the least amount of energy to produce heat. Solar powered heaters are the most sustainable option as they rely on renewable energy from the sun. Electric resistance heaters are also quite sustainable as they do not produce any emissions. Heat pumps are another efficient and sustainable option for heating your home or office space. Geothermal systems are yet another type of sustainable heater as they use the earth's natural heat to warm your space. When choosing a sustainable heater, it is important to consider the upfront cost, operating cost, and maintenance cost over the lifetime of the heater. Make sure the heater is properly sized for the space it will be used in and place it in a central location so that it can evenly heat the space. Insulate the wall and ceiling around the heater to help keep heat in. Use drapery or blinds to cover windows and help keep heat in. Keep doors closed 

In [84]:
# attempt2 to get keyword form summary (final)
ans_key_summary = []

response = openai.Completion.create(
    engine="text-davinci-002",
    prompt="Extract keywords from this text:\n\n{}".format(response_text_summary),
    temperature=0,
    max_tokens=100,
    top_p=1,
    frequency_penalty=0.7,
    presence_penalty=1
    )
response_text = response.choices[0]['text']
response_text = response_text.replace("\n", "").replace("  ", "")
ans_key_summary.append(response_text)
ans_key_summary



ans_key3_clean2 = []
for a_string in ans_key_summary:
    a_list = a_string.split(', ')
   
    for a in a_list:
        if a not in qns_key3:
            if 'efficien' not in a and 'sustain' not in a and 'eater' not in a:
                ans_key3_clean2.append(a)
print('from: ',ans3)
print('to: ',ans_key3_clean2)


from:  ['sustainability is important when designing a heater,the most efficient heater are those that use the least amount of energy to produce heat,solar powered heater are the most sustainable option a they rely on renewable energy from the sun,electric resistance heater are also quite sustainable a they do not produce any emission,heat pump are another efficient and sustainable option for heating your home or office space,geothermal system are yet another type of sustainable heater a they use the earth is natural heat to warm your space,when choosing a sustainable heater it is important to consider the upfront cost operating cost and maintenance cost over the', 'make sure the heater is properly sized for the space it will be used in,place the heater in a central location so that it can evenly heat the space,insulate the wall and ceiling around the heater to help keep heat in,use drapery or blind to cover window and help keep heat in,keep door closed to room that are not being heated

In [86]:
# show all keywords
print('keywords for sustainable design and metrics')
print(ans_key_clean)
print('keywords for functions and behaviour design')
print(ans_key2_clean)
print('keywords for sustainable design')
print(ans_key3_clean2)

keywords for sustainable design and metrics
['btu', 'emission output', 'carbon dioxide equivalent', 'water conservation', 'gallon per minute', 'material reuse', 'renewable', 'source', 'solar', 'wind', 'power', 'thermal mass', 'fuel', 'appliance', 'space', 'heated']
keywords for functions and behaviour design
['timer', 'on', 'off.', 'behavioral property', 'easy to use', 'control', 'feedback', 'temperature', 'setting', 'furnace', 'hot air', 'warm', 'interior', 'home.']
keywords for sustainable design
['solar powered', 'electric resistance', 'heat pumps', 'geothermal systems', 'upfront cost', 'operating cost', 'maintenance cost']


In [89]:
# Contatenate all AI suggestions 
a = ','.join([a.replace('however ','') for a in ans])
b =','.join([a.replace('however ','') for a in ans2])
c = ','.join(response_text_summary.split('.'))
print(a ,'\n',b,'\n', c)

d = a+b+c


some possible metric that could be used to design a sustainable heater include energy efficiency measured in term of british thermal unit or btu emission output measured in term of carbon dioxide equivalent or co2e water conservation measured in gallon per minute or gpm and material reuse recycling potential,some tip for designing a sustainable heater include using renewable energy source such a solar or wind power incorporating thermal mass into the design to help store heat and using insulation to minimize heat loss,the type of fuel being used e g natural gas electricity propane the efficiency rating of the appliance and how well insulated the space is that need to be heated 
 the user should also be able to easily set the timer for when the heater will turn on and off,some tip for designing a heater with good behavioral property include making sure that the heater is easy to use and control providing clear feedback to the user about the current temperature and setting and incorporat

In [93]:
# Summary for designing for sustainable heater
response = openai.Completion.create(
  engine="text-davinci-002",
  prompt= "when designing a heater" + d + "\n\nTl;dr:\n\n",
  temperature=0,
  max_tokens=250,
  top_p=1,
  frequency_penalty=1,
  presence_penalty=1
)
final_suggestion= response.choices[0]['text'].replace("\n", "")
print('final suggestion: ',final_suggestion)

final suggestion:  When designing a heater, it is important to consider sustainability. The most efficient heaters are those that use the least amount of energy to produce heat. Solar powered heaters are the most sustainable option as they rely on renewable energy from the sun. Electric resistance heaters are also quite sustainable as they do not produce any emissions. Heat pumps are another efficient and sustainable option for heating your home or office space. Geothermal systems are yet another type of sustainable heater as they use the earth's natural heat to warm your space.


In [107]:
# extract only noun keywords
import spacy
nlp = spacy.load("en_core_web_sm")
akc =[]
ak2c = []
ak3c = []
ak3c2 = []
# taking input

for key in ans_key_clean:
    # returns a document of object
    doc = nlp(key)

    # checking if it is a noun or not
    for i in range(len(doc)):
        if(doc[i].tag_ == 'NNP' or doc[i].tag_ == 'NN'):
            # print(doc[i])
            # print(key, " is a noun.")
            akc.append(key)
            break
        else:
            # print(doc[i], " is not a noun.")
            pass

for key in ans_key2_clean:
    # returns a document of object
    doc = nlp(key)

    # checking if it is a noun or not
    for i in range(len(doc)):
        if(doc[i].tag_ == 'NNP' or doc[i].tag_ == 'NN'):
            # print(doc[i])
            # print(key, " is a noun.")
            ak2c.append(key)
            break
        else:
            # print(doc[i], " is not a noun.")
            pass

for key in ans_key3_clean:
    # returns a document of object
    doc = nlp(key)

    # checking if it is a noun or not
    for i in range(len(doc)):
        if(doc[i].tag_ == 'NNP' or doc[i].tag_ == 'NN'):
            # print(doc[i])
            # print(key, " is a noun.")
            ak3c.append(key)
            break
        else:
            # print(doc[i], " is not a noun.")
            pass

for key in ans_key3_clean2:
    # returns a document of object
    doc = nlp(key)

    # checking if it is a noun or not
    for i in range(len(doc)):
        if(doc[i].tag_ == 'NNP' or doc[i].tag_ == 'NN'):
            # print(doc[i])
            # print(key, " is a noun.")
            ak3c2.append(key)
            break
        else:
            # print(doc[i], " is not a noun.")
            pass
print('final keywords for sustainable design and metrics')        
print(akc)
print(ak2c)
print(ak3c)
print(ak3c2)

final keywords for sustainable design and metrics
['btu', 'emission output', 'carbon dioxide equivalent', 'water conservation', 'gallon per minute', 'material reuse', 'source', 'wind', 'power', 'thermal mass', 'fuel', 'appliance', 'space']
['timer', 'behavioral property', 'control', 'feedback', 'temperature', 'furnace', 'hot air', 'interior', 'home.']
['solar powered', 'electric resistance', 'emission', 'heat pump', 'geothermal system', 'size', 'space', 'central location', 'evenly heat', 'wall', 'ceiling', 'drapery', 'window', 'door closed', 'space', 'climate', 'building material', 'factor', 'size', 'capacity', 'location']
['solar powered', 'electric resistance', 'heat pumps', 'upfront cost', 'operating cost', 'maintenance cost']


In [110]:
# ads ( just for fun :D )

response = openai.Completion.create(
  engine="text-davinci-002",
  prompt="Write a creative ad for the following product to run on Facebook aimed at parents:\n\nProduct:" + ''.join(ans3) ,
  temperature=0,
  max_tokens=1000,
  top_p=1,
  frequency_penalty=1,
  presence_penalty=1
)
response_text_ads = response.choices[0]['text']
response_text_ads = response_text_ads.replace("\n", " ")
print('HOW TO MARKET YOUR HEATER for parent:')
response_text_ads


HOW TO MARKET YOUR HEATER for parent:


' take measures to help keep heat in the space such as insulating the walls and ceiling around the heater,using drapery or blinds to cover windows and keeping doors closed to rooms that are not being heated,finally setting the thermostat at a comfortable temperature and only using the heater when necessary can also help to improve it efficiency  Looking for an efficient and sustainable way to heat your home or office? Look no further than solar powered heaters! Solar powered heaters rely on renewable energy from the sun, making them a sustainable option for heating your space. Electric resistance heaters are also quite sustainable as they do not produce any emissions. Heat pumps are another efficient and sustainable option for heating your home or office space. Geothermal systems are yet another type of sustainable heater as they use the earth’s natural heat to warm your space. When choosing a sustainable heater, it is important to consider the upfront cost, operating cost, and mainten

#### some other exploration (WIP)

In [106]:
import spacy

nlp = spacy.load("en_core_web_sm")

listToStr = " ".join(ans_key).replace(",", " ").replace(".", " ").replace("  ", " ").replace("   ", " ")
print(listToStr)
simi = []
tokens = nlp(listToStr) 

for idx, tok in enumerate(tokens):
    sim = []
    for nextok in tokens[idx:]:
        sim.append([tok.similarity(nextok)])
    simi.append(sim)


energy efficiency btu emission output carbon dioxide equivalent water conservation gallon per minute material reuse sustainable heater design renewable energy source solar wind power thermal mass fuel efficiency appliance space heated


C:\Users\Pin Nean\AppData\Local\Temp\ipykernel_25348\3044207164.py:14: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  sim.append([tok.similarity(nextok)])


In [14]:
simi

[[[1.0],
  [0.3962400555610657],
  [0.2925609350204468],
  [0.09922326356172562],
  [0.04566784203052521],
  [0.040009573101997375],
  [-0.028609683737158775],
  [1.0],
  [0.22849442064762115],
  [0.11409429460763931],
  [0.0033480117563158274],
  [0.15940342843532562],
  [0.09690920263528824],
  [0.08525314182043076],
  [0.2615750730037689],
  [0.05173901841044426],
  [0.04591098800301552],
  [0.11865878850221634],
  [-0.003725723596289754],
  [-0.02574589103460312],
  [1.0],
  [0.310879647731781],
  [-0.017737815156579018],
  [0.06704889237880707],
  [-0.11246155947446823],
  [-0.06995963305234909]],
 [[1.0],
  [0.48242029547691345],
  [0.5499566197395325],
  [0.3358232080936432],
  [0.14494343101978302],
  [1.0],
  [0.19992651045322418],
  [0.2165025770664215],
  [0.43101972341537476],
  [0.4137245714664459],
  [0.10401561856269836],
  [1.0],
  [0.34076055884361267],
  [0.26349976658821106],
  [0.4543159306049347],
  [0.18164271116256714],
  [0.11265857517719269],
  [0.1286784261465